In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_chroma import Chroma

from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [3]:
# 출력 예쁘게 하기
from rich.console import Console
from rich.table import Table

console = Console()

def rich_docs(docs, title="Retriever Results", max_len=140):
    table = Table(title=title)
    table.add_column("#", justify="right")
    table.add_column("Source")
    table.add_column("Page", justify="right")
    table.add_column("Preview")

    for i, d in enumerate(docs, 1):
        m = d.metadata or {}
        src = (m.get("source","") or "").split("/")[-1]
        page = str(m.get("page_label", m.get("page",0)+1))
        text = (d.page_content or "").strip().replace("\n", " ")
        content = (text[:max_len] + ("…" if len(text) > max_len else ""))
        table.add_row(str(i), src, page, content)

    console.print(table)

# 1. 검색기 

## 1. 벡터 스토어 가져오기

In [4]:
# 벡터저장소가 이미 있는 상황

embedding = OpenAIEmbeddings(
    model="text-embedding-3-small"
)
persist_dir = "../vectorstore\chromadb_advanced_store"
collection_name = "samsung"

vectorsore = Chroma(
    persist_directory=persist_dir,
    collection_name=collection_name,
    embedding_function=embedding
)

## 2. retriever 만들기

In [6]:
retriever = vectorsore.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k" : 30}
)
question = "삼성과 현대 중에서 어느 기업이 더 나아?"
result = retriever.invoke(question)
rich_docs(result, title='기업 비교')

                                                     기업 비교                                                     
┏━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃  # ┃ Source                                           ┃ Page ┃ Preview                                          ┃
┡━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│  1 │ Samsung_Electronics_Sustainability_Report_2025_… │    5 │ 삼성전자 지속가능경영보고서 2025 05 Our Company  │
│    │                                                  │      │ AppendixFacts & Figures PrinciplePlanet People   │
│    │                                                  │      │ 회사소개 About Us 삼성전자주식회사(이하          │
│    │                                                  │      │ 삼성전자)는 인재와 기술을 기반으로 최고의 제품 … │
│    │                                                  │      │ 서비스를…                                        │
│  2 │ Samsung_Electronics_Sustainability_Report_2025_… │    3 │ 삼성전자 지속가능경영보고서 2025 03 Our Company  │
│    │                                                  │      │ AppendixFacts & Figures PrinciplePlanet People   │
│    │                                                  │      │ Our Company CEO 메시지 회사소개 기업 지배구조    │
│    │                                                  │      │ 중대성 평가 이해관계자 소통 04 05 06 …           │
│  3 │ Samsung_Electronics_Sustainability_Report_2025_… │    4 │ 삼성전자 지속가능경영보고서 2025 04 Our Company  │
│    │                                                  │      │ AppendixFacts & Figures PrinciplePlanet People   │
│    │                                                  │      │ 주주, 고객, 협력회사, 그리고 임직원 여러분,      │
│    │                                                  │      │ 2024년은 글로벌 지정학적 리스크와 AI 기술의 성 … │
│    │                                                  │      │ …                                                │
│  4 │ Samsung_Electronics_Sustainability_Report_2025_… │   46 │ 정기적으로 모니터링하고 시스템으로 관리합니다.   │
│    │                                                  │      │ 하위 공급망 관리 지속적으로 증가하는 공급망      │
│    │                                                  │      │ 변동성과 불 확실성에 대응하기 위하여 삼성전자는  │
│    │                                                  │      │ 1 차 협력회사뿐 아니라 주요  품목을 공급하는     │
│    │                                                  │      │ 하위 협력회사에 대한 정보 관리를 점진적으로      │
│    │                                                  │      │ 확대하고 있습니다.…                              │
│  5 │ Samsung_Electronics_Sustainability_Report_2025_… │   50 │ 삼성전자 지속가능경영보고서 2025 50 안전보건     │
│    │                                                  │      │ 삼성전자는 DX부문의 Global EHS실장, DS부문의     │
│    │                                                  │      │ 글로벌 제조&인프라  총괄장인 CSO(Chief Safety    │
│    │                                                  │      │ Officer)를 중심으로 2030년까지 상주  협력회사    │
│    │                                                  │      │ 안전보건 역량과 관리체계 …                       │
│  6 │ Samsung_Electronics_Sustainability_Report_2025_… │   54 │ 삼성전자 지속가능경영보고서 2025 54 Bespoke AI   │
│    │                                                  │      │ 로봇청소기 보안 인증 획득 2024년에는 Bespoke AI  │
│    │                                                  │      │ 스팀 로봇청소기가 KISA 개인정보보호중심설계      │
│    │                                                  │      │ (PbD, Privacy by Design) 인증과 KISA IoT 보안    │
│    │                                                  │      │ 인증 중 …                                        │
│  7 │ Samsung_Electronics_Sustainability_Report_2025_… │   45 │ 협력회사를 지원하고 교육합니다.  협력회사의      │
│    │                                                  │      │ 지속가능경영 관리 수준 향상을 위한 실사와 개     │
│    │                                                  │      │ 선조치 이행, 정보공개 등의 성과는 해 당 부서의   │
│    │                                                  │      │ 경영진과 실무자들의 KPI로 반영되며 목표 달성은   │
│    │                            

## 3. reranker 만들기

In [7]:
from langchain_community.cross_encoders.huggingface import HuggingFaceCrossEncoder
from langchain.retrievers.document_compressors import CrossEncoderReranker

hf_ce = HuggingFaceCrossEncoder(
    model_name = "cross-encoder/ms-marco-MiniLM-L6-v2",
    model_kwargs = {
        "device" : "cuda",
        "max_length" : 512
    }
)

reranker = CrossEncoderReranker(
    model = hf_ce,
    top_n = 10
)

## 4. retriever -> reranker

In [8]:
from langchain.retrievers import ContextualCompressionRetriever

com_retriever = ContextualCompressionRetriever(
    base_retriever=retriever,
    base_compressor=reranker
)

## 5. reorder(순서 정리)

In [9]:
from langchain_community.document_transformers import LongContextReorder

reorder = LongContextReorder()

## 6. 검색 결과 문서 합치는 함수 생성

In [10]:
# 문서를 합치는 함수
def format_docs(docs):
    return "\n\---\n\n".join([doc.page_content for doc in docs])

# 2. 기본 체인 만들기

## 1) 프롬프트 설정

In [11]:
reg_prompt = ChatPromptTemplate.from_messages([
    ('system', """주어진 컨텍스트만 근거로 간결하고 정화하게 답하도록 해라
     
     [컨텍스트]
     {context}
    """),
    ('user', "{question}")
])

## 2) 모델 설정

In [12]:
model = ChatOpenAI(
    temperature=0,
    model = "gpt-4.1-mini"
)

## 3) outputparser

In [13]:
outputparser = StrOutputParser()

In [14]:
chain = reg_prompt | model | outputparser

# 3. 통합체인 만들기

In [18]:
rag_chain = (
    {
        'docs' : RunnableLambda(lambda x: com_retriever.invoke(x['question'])), # reranker
        'question' : RunnablePassthrough()
    }
    | RunnableLambda(lambda x : {
        "context" : format_docs(reorder.transform_documents(x['docs'])), # reorder
        'question' : x['question']
    })
    | chain
)

rag_chain.invoke({
    'question' : "삼성의 미래 계획은 어떻게 되나요?"
})

'삼성전자는 인재와 기술을 바탕으로 최고의 제품과 서비스를 창출하여 인류사회에 공헌한다는 경영철학 아래, 기술 리더십을 강화하고 새로운 영역에서 미래 성장동력을 확보하는 데 주력할 계획입니다. 또한, 지속가능경영 관리체계와 전략을 충실히 이행하며, 이해관계자 의견을 반영해 지속가능한 성장 기반을 마련하는 데 최선을 다할 예정입니다.'

In [26]:
rag_chain = (
    {
        'docs' : RunnableLambda(lambda x: com_retriever.invoke(x['question'])), # reranker
        'question' : RunnablePassthrough()
    }
    | RunnableLambda(lambda x : {
        "docs" : reorder.transform_documents(x['docs']), # reorder
        'question' : x['question']
    })
    | 
    {
        "context" : RunnableLambda(lambda x : format_docs(x['docs'])),
        'question' : RunnablePassthrough()
    }
    | chain
)

rag_chain.invoke({
    'question' : "삼성의 미래 계획은 어떻게 되나요?"
})

'삼성전자는 ‘함께가요 미래로! Enabling People’ 비전 아래, 청소년에게 양질의 교육 기회를 제공하고 중소기업과 스타트업에 경영 노하우를 전수하는 프로그램을 운영합니다. 2025년에는 삼성 청년SW·AI아카데미 교육 기회를 마이스터고 졸업생까지 확대하고, 자립준비 청년 지원을 위한 희망디딤돌 인천센터를 추가 설립할 예정입니다. 또한, 지속가능경영위원회와 ESG경영협의회를 통해 환경경영전략을 수립·관리하며, 2050년 탄소중립 달성, 재생에너지 전환, 안전보건 강화 등 중장기 목표를 추진합니다. 기술 리더십을 바탕으로 미래 성장동력을 확보하고 지속가능한 성장 기반 마련에 최선을 다할 계획입니다.'

In [23]:
rag_chain = (
    {
        "context" : RunnableLambda(lambda x: x['question']) | com_retriever | format_docs,
        "question" : RunnablePassthrough()
    } 
    | chain
)

rag_chain.invoke({
    'question' : "삼성의 미래 계획은 어떻게 되나요?"
})

'삼성전자는 인재와 기술을 바탕으로 최고의 제품과 서비스를 창출하여 인류사회에 공헌한다는 경영철학 아래, 기술 리더십으로 재도약의 기반을 다지고 새로운 영역에서 미래 성장동력을 확보해 나갈 계획입니다. 또한, 지속가능한 성장 기반 마련을 위해 이해관계자의 의견에 귀 기울이며 지속적으로 노력할 예정입니다.'

In [24]:
rag_chain = (
    {
        "context" : RunnableLambda(lambda x: x['question']) | com_retriever | RunnableLambda(lambda x: reorder.transform_documents(x)) | format_docs,
        "question" : RunnablePassthrough()
    } 
    | chain
)

rag_chain.invoke({
    'question' : "삼성의 미래 계획은 어떻게 되나요?"
})

'삼성전자는 인재와 기술을 바탕으로 최고의 제품과 서비스를 창출하여 인류사회에 공헌한다는 경영철학 아래, 기술 리더십을 강화하고 새로운 영역에서 미래 성장동력을 확보하는 데 주력할 계획입니다. 또한, 지속가능경영 관리체계와 전략을 충실히 이행하며, 이해관계자의 의견을 반영해 지속가능한 성장 기반을 마련하는 데 최선을 다할 예정입니다.'

# 4. multi input chain

In [29]:
com_retriever_chain = RunnableLambda(lambda x: x['question']) | com_retriever | format_docs

In [30]:
# 1. 프롬프트 설정
rag_prompt = ChatPromptTemplate.from_messages([
    ("system", """주어진 컨텍스트만 근거로 간결하고 정확하게 답하도록 해라. 
     
     [컨텍스트]
     {context}
     
     """),
    ("human", "{pro} 스타일에 맞게 {question}에 대답해라")
])

# 2. 모델 설정
model = ChatOpenAI(
    model = "gpt-4.1-mini",
    temperature=0
)

# 3. outputparser 
outputparser = StrOutputParser()

# 4. 체인 설정
chain = rag_prompt | model | outputparser
chain

ChatPromptTemplate(input_variables=['context', 'pro', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='주어진 컨텍스트만 근거로 간결하고 정확하게 답하도록 해라. \n\n     [컨텍스트]\n     {context}\n\n     '), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['pro', 'question'], input_types={}, partial_variables={}, template='{pro} 스타일에 맞게 {question}에 대답해라'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001EE868BDDD0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001EE86755D50>, root_client=<openai.OpenAI object at 0x000001EE868042D0>, root_async_client=<openai.AsyncOpenAI object at 0x000001EE868BCC50>, model_name='gpt-4.1-mini', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)
|

In [31]:
rag_chain = (
    {
        "context" :  com_retriever_chain,
        "question" : RunnableLambda(lambda x: x['question']),
        "pro" : RunnableLambda(lambda x: x['pro'])
    }
    | chain
)
rag_chain.invoke({
    "question" : "삼성의 미래 계획은 어떻게 되나요?",
    "pro" : "냥냥체"
})

'냥냥~ 삼성은 2050년까지 탄소중립을 목표로 기후변화에 맞서고 있냥! 재생에너지 전환도 쭉쭉 늘리고, 물 사용도 아껴서 환경을 지키려 노력 중이냥~ 또, 청년들 교육에 힘쓰고, 중소기업과 스타트업도 활짝 지원해서 모두 함께 미래로 달려가려 한다옹! 미래 준비는 삼성과 함께라냥~!'